In [2]:
from oauth2client.client import GoogleCredentials
from googleapiclient import discovery
from googleapiclient import errors
import tensorflow as tf
import tensorflow_recommenders as tfrs
import pandas as pd
import numpy as np
import json
import os

from tqdm.auto import tqdm
tqdm.pandas(leave=False)

# Training

## configuration

In [3]:
PROJECT                  = "mlteam-ml-specialization-2021"
TRAIN_PATH               = "gs://mlteam-ml-specialization-2021-blackfriday/dataset/parsed/202104130952/train.csv"
EVAL_PATH                = "gs://mlteam-ml-specialization-2021-blackfriday/dataset/parsed/202104130952/test/evalset.csv"
TEST_PATH                = "gs://mlteam-ml-specialization-2021-blackfriday/dataset/parsed/202104130952/test/testset.csv"
MODEL_NAME               = "008_hptuning_product_features"
BATCH_SIZE               = 30000 #1 epoch -> 3 steps
NUM_EPOCHS               = 500
SCANN_NUM_NEIGHBORS      = 100
LEARNING_RATE            = 0.01
EMBEDDING_DIM            = 128
USER_FEATURES            = ["Gender", "Age", "Occupation", "City_Category", "Stay_In_Current_City_Years", "Marital_Status"]
USER_INPUT_EMBEDDING_DIM = 8
FORCE                    = False

In [9]:
%%writefile config.yaml
trainingInput:
  hyperparameters:
    goal: MINIMIZE
    hyperparameterMetricTag: epoch_total_loss
    maxTrials: 8
    maxParallelTrials: 8
    enableTrialEarlyStopping: False
    params:
    - parameterName: product-input-embedding-dim
      type: DISCRETE
      discreteValues:
      - 16
      - 128
    - parameterName: embedding-dim
      type: DISCRETE
      discreteValues:
      - 128
    - parameterName: common-layers
      type: CATEGORICAL
      categoricalValues:
      - '[256]'
      - '[512,256,128]'
    - parameterName: product-features
      type: CATEGORICAL
      categoricalValues:
      - '["Product_Category_1"]'
      - '["Product_ID", "Product_Category_1"]'

Writing config.yaml


In [10]:
MODEL_PATH = os.path.join("gs://mlteam-ml-specialization-2021-blackfriday/aiplatform_jobs", MODEL_NAME)
if tf.io.gfile.exists(MODEL_PATH):
    if FORCE:
        tf.io.gfile.rmtree(MODEL_PATH)
    else:
        assert not tf.io.gfile.exists(MODEL_PATH), f"{MODEL_PATH} already exists, set FORCE=True to overwrite"


os.environ["TRAIN_PATH"]               = str(TRAIN_PATH)
os.environ["EVAL_PATH"]                = str(EVAL_PATH)
os.environ["TEST_PATH"]                = str(TEST_PATH)
os.environ["MODEL_NAME"]               = str(MODEL_NAME)
os.environ["MODEL_PATH"]               = str(MODEL_PATH)
os.environ["BATCH_SIZE"]               = str(BATCH_SIZE)
os.environ["NUM_EPOCHS"]               = str(NUM_EPOCHS)
os.environ["SCANN_NUM_NEIGHBORS"]      = str(SCANN_NUM_NEIGHBORS)
os.environ["LEARNING_RATE"]            = str(LEARNING_RATE)
os.environ["EMBEDDING_DIM"]            = str(EMBEDDING_DIM)
os.environ["USER_INPUT_EMBEDDING_DIM"] = str(USER_INPUT_EMBEDDING_DIM)
os.environ["USER_FEATURES"]            = json.dumps(USER_FEATURES)

## launch

In [11]:
%%bash

# JOB_NAME: the name of your job running on AI Platform.
JOB_NAME=bf_${MODEL_NAME}_$(date +%Y%m%d_%H%M%S)

# REGION: select a region from https://cloud.google.com/ai-platform/training/docs/regions
# or use the default '`us-central1`'. The region is where the model will be deployed.
REGION=europe-west1
PYTHON_VERSION=3.7
RUNTIME_VERSION=2.4

current_dir="$(cd "$(dirname "${BASH_SOURCE[0]}")" >/dev/null 2>&1 && pwd)"
cd ${current_dir}"/../../.."

config_file="${current_dir}/config.yaml"

gcloud ai-platform jobs submit training "${JOB_NAME}" \
  --package-path trainer/ \
  --module-name trainer.task \
  --region ${REGION} \
  --python-version ${PYTHON_VERSION} \
  --runtime-version ${RUNTIME_VERSION} \
  --job-dir "${MODEL_PATH}" \
  --config "${config_file}" \
  -- \
  --train-path="${TRAIN_PATH}" \
  --eval-path="${EVAL_PATH}" \
  --job-dir="${MODEL_PATH}" \
  --num-epochs=${NUM_EPOCHS} \
  --batch-size=${BATCH_SIZE} \
  --scann-num-neighbors=${SCANN_NUM_NEIGHBORS} \
  --user-features="${USER_FEATURES}" \
  --embedding-dim=${EMBEDDING_DIM} \
  --learning-rate=${LEARNING_RATE} \
  --user-input-embedding-dim=${USER_INPUT_EMBEDDING_DIM}


gcloud ai-platform jobs describe ${JOB_NAME}

jobId: bf_008_hptuning_product_features_20210601_171319
state: QUEUED
createTime: '2021-06-01T17:13:22Z'
etag: E-xMP98avl0=
jobId: bf_008_hptuning_product_features_20210601_171319
state: PREPARING
trainingInput:
  args:
  - --train-path=gs://mlteam-ml-specialization-2021-blackfriday/dataset/parsed/202104130952/train.csv
  - --eval-path=gs://mlteam-ml-specialization-2021-blackfriday/dataset/parsed/202104130952/test/evalset.csv
  - --job-dir=gs://mlteam-ml-specialization-2021-blackfriday/aiplatform_jobs/008_hptuning_product_features
  - --num-epochs=500
  - --batch-size=30000
  - --scann-num-neighbors=100
  - --user-features=["Gender", "Age", "Occupation", "City_Category", "Stay_In_Current_City_Years",
    "Marital_Status"]
  - --embedding-dim=128
  - --learning-rate=0.01
  - --user-input-embedding-dim=8
  hyperparameters:
    goal: MINIMIZE
    hyperparameterMetricTag: epoch_total_loss
    maxParallelTrials: 8
    maxTrials: 8
    params:
    - discreteValues:
      - 16.0
      - 128.0

Job [bf_008_hptuning_product_features_20210601_171319] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe bf_008_hptuning_product_features_20210601_171319

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs bf_008_hptuning_product_features_20210601_171319

View job in the Cloud Console at:
https://console.cloud.google.com/mlengine/jobs/bf_008_hptuning_product_features_20210601_171319?project=mlteam-ml-specialization-2021

View logs at:
https://console.cloud.google.com/logs?resource=ml_job%2Fjob_id%2Fbf_008_hptuning_product_features_20210601_171319&project=mlteam-ml-specialization-2021


In [12]:
!rm -rf config.yaml

In [4]:
metadata_path = os.path.join("gs://mlteam-ml-specialization-2021-blackfriday/aiplatform_jobs", MODEL_NAME, "metadata.json")

ml               = discovery.build('ml','v1')
projectId        = 'projects/{}'.format(PROJECT)
ai_platform_data = ml.projects().jobs().list(parent=projectId).execute()
jobs             = ai_platform_data["jobs"]
latest_job       = sorted([j for j in jobs if j['jobId'].startswith(f"bf_{MODEL_NAME}")], key=lambda x: x["jobId"])[-1]
if latest_job["trainingOutput"].get('isHyperparameterTuningJob',None) is not None:
    trials = latest_job["trainingOutput"]["trials"]
    json.dump(trials, tf.io.gfile.GFile(metadata_path, "w"))
else:
    metadata = [{
        'trialId'        : '1',
        'hyperparameters': {},
        'startTime'      : latest_job["startTime"],
        'endTime'        : latest_job["endTime"],
        'state'          : latest_job["state"],
    }]
    json.dump(metadata, tf.io.gfile.GFile(metadata_path, "w"))

# Evaluation

## ground truth

In [5]:
ALL_USER_FEATURES   = ["Gender", "Age", "Occupation", "City_Category", "Stay_In_Current_City_Years", "Marital_Status"] 

df = pd.read_csv(TEST_PATH)

## baseline

In [6]:
class BlackFridayBaseline:
    def __init__(self, trainset_location, user_features_list, build_inplace=False):
        self.trainset_location, self.user_features_list = trainset_location, user_features_list
        if build_inplace:
            self.build()
        
    def build(self):
        self.df = pd.read_csv(self.trainset_location).groupby(
            self.user_features_list+["Product_ID"])["User_ID"].count().reset_index().rename(
            columns={"User_ID":"count_product"})
        
    def predict(self, user_features, n_products):
        mask = None
        for k,v in user_features.items():
            if mask is None:
                mask = self.df[k]==v
            else:
                mask = mask&(self.df[k]==v)
        return self.df[mask].sort_values(self.user_features_list+["count_product"])["Product_ID"].values[:n_products]
        

## model(s)

In [7]:
class PredictionModel:
    def __init__(self, model_path, build_inplace=False):
        self.model_path=model_path
        if build_inplace:
            self.build()
    
    def build(self):
        self.model=tf.keras.models.load_model(self.model_path)
        
    def predict_batch(self, model_input, n_products):
        scores, products = self.model({k:tf.constant(v) for k,v in model_input.items()})
        return products.numpy()[:,:n_products]
    
    def predict_single(self, model_input, n_products):
        scores, products = self.model({k:tf.constant([v]) for k,v in model_input.items()})
        return products.numpy()[0,:n_products]
    


    

## stats

In [8]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
df_grouped = pd.read_csv(TEST_PATH).groupby(ALL_USER_FEATURES)["Product_ID"].apply(list).apply(np.array).reset_index()
top_k = [1, 10, 50, 100]
results = dict()

baseline = BlackFridayBaseline(TRAIN_PATH, ALL_USER_FEATURES, True)
for k in tqdm(top_k, leave=False):
    metrics_id = f"baseline_top{k}"
    df_grouped[metrics_id]         = df_grouped[ALL_USER_FEATURES].progress_apply(lambda x: baseline.predict(x, k), axis=1)
    baseline_true_positives        = df_grouped.progress_apply(lambda x: np.intersect1d(x["Product_ID"],x[metrics_id]).shape[0], axis=1).sum()
    baseline_false_negatives       = df.shape[0] - baseline_true_positives
    baseline_false_positives       = df_grouped[metrics_id].apply(lambda x: x.shape[0]).sum() - baseline_true_positives
    baseline_reach                 = df_grouped.progress_apply(lambda x: min(x["Product_ID"].shape[0],x[metrics_id].shape[0]), axis=1).sum()
    results[f"baseline_top{k}"]    = {
        "top"  : k,
        "model": "baseline",
        "tp"   : baseline_true_positives,
        "fp"   : baseline_false_positives,
        "fn"   : baseline_false_negatives,
        "reach": baseline_reach #portata
    }
    

metadata_path = os.path.join("gs://mlteam-ml-specialization-2021-blackfriday/aiplatform_jobs", MODEL_NAME, "metadata.json")
metadata = json.load(tf.io.gfile.GFile(metadata_path, "r"))
for j in tqdm(metadata, leave=False):
    model = PredictionModel(os.path.join("gs://mlteam-ml-specialization-2021-blackfriday/aiplatform_jobs", MODEL_NAME, j['trialId'], 'Scann'), True)
    hps = json.dumps(j["hyperparameters"])
    model_id = f"trial_{j['trialId']}"
    for k in tqdm(top_k, leave=False):
        metrics_id=f"trial_{j['trialId']}_top{k}"
        df_grouped[metrics_id] = df_grouped[USER_FEATURES].progress_apply(lambda x: model.predict_single(x.astype(str), k).astype(str), axis=1)
        tp = df_grouped.progress_apply(lambda x: np.intersect1d(x["Product_ID"],x[metrics_id]).shape[0], axis=1).sum()
        fn = df.shape[0] - tp
        fp = df_grouped[metrics_id].apply(lambda x: x.shape[0]).sum() - tp
        reach = df_grouped.progress_apply(lambda x: min(len(x["Product_ID"]),x[metrics_id].shape[0]), axis=1).sum() 
        results[metrics_id] = {
            "top"  : k,
            "model": model_id,
            "tp"   : tp,
            "fp"   : fp,
            "fn"   : fn,
            "reach": reach
        }
    
df_results = pd.DataFrame(results).T
df_results["precision"] = df_results["tp"]/(df_results["tp"]+df_results["fp"])
df_results["recall"] = df_results["tp"]/(df_results["tp"]+df_results["fn"])
df_results["tp_over_reach"] = df_results["tp"]/df_results["reach"]
df_results

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

...

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

  0%|          | 0/2430 [00:00<?, ?it/s]

,top,model,tp,fp,fn,reach,precision,recall,tp_over_reach
baseline_top1,1,baseline,637,1754,273647,2391,0.266416,0.002322,0.266416
baseline_top10,10,baseline,5184,14437,269100,19618,0.264207,0.0189,0.264247
baseline_top50,50,baseline,15198,39057,259086,54252,0.280122,0.05541,0.280137
baseline_top100,100,baseline,19780,47500,254504,67277,0.293995,0.072115,0.294008
trial_8_top1,1,trial_8,84,2346,274200,2430,0.034568,0.000306,0.034568
trial_8_top10,10,trial_8,902,23398,273382,23612,0.037119,0.003289,0.038201
trial_8_top50,50,trial_8,5855,115645,268429,91899,0.048189,0.021346,0.063711
trial_8_top100,100,trial_8,13546,229454,260738,146329,0.055745,0.049387,0.092572
trial_7_top1,1,trial_7,181,2249,274103,2430,0.074486,0.00066,0.074486
trial_7_top10,10,trial_7,1668,22632,272616,23612,0.068642,0.006081,0.070642


In [9]:
df_results.pivot("model","top","precision")

top,1,10,50,100
model,,,,
baseline,0.266416,0.264207,0.280122,0.293995
trial_1,0.047325,0.051852,0.061078,0.06528
trial_2,0.076543,0.08107,0.070189,0.060403
trial_3,0.074897,0.084856,0.071037,0.060868
trial_4,0.061317,0.057119,0.068807,0.073066
trial_5,0.076543,0.082634,0.074362,0.066226
trial_6,0.032922,0.035391,0.042214,0.047609
trial_7,0.074486,0.068642,0.05744,0.052691
trial_8,0.034568,0.037119,0.048189,0.055745


In [10]:
df_results.pivot("model","top","recall")

top,1,10,50,100
model,,,,
baseline,0.002322,0.0189,0.05541,0.072115
trial_1,0.000419,0.004594,0.027056,0.057834
trial_2,0.000678,0.007182,0.031092,0.053514
trial_3,0.000664,0.007518,0.031467,0.053926
trial_4,0.000543,0.00506,0.030479,0.064732
trial_5,0.000678,0.007321,0.03294,0.058673
trial_6,0.000292,0.003135,0.0187,0.042179
trial_7,0.00066,0.006081,0.025444,0.046682
trial_8,0.000306,0.003289,0.021346,0.049387


In [11]:
df_results.pivot("model","top","tp_over_reach")

top,1,10,50,100
model,,,,
baseline,0.266416,0.264247,0.280137,0.294008
trial_1,0.047325,0.053363,0.080752,0.108406
trial_2,0.076543,0.083432,0.092798,0.100308
trial_3,0.074897,0.087328,0.093918,0.10108
trial_4,0.061317,0.058784,0.090969,0.121336
trial_5,0.076543,0.085042,0.098314,0.109978
trial_6,0.032922,0.036422,0.055811,0.079062
trial_7,0.074486,0.070642,0.075942,0.087501
trial_8,0.034568,0.038201,0.063711,0.092572
